In [ ]:
import os
import sys
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_7.ipynb')
git_add(file_path)
git_commit('Updated test 7')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

True

In [ ]:
model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0.5
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly AI assistant."),
    
    ("human", "{input}")
])

memory = ConversationBufferMemory(
    memory_key = "chat_history"
)

chain = prompt | model

msg = {
    "input": "Hello, how are you?"
}

response = chain.invoke(
    msg
)

print(response)

content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 24, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-51c212ab-1afb-4680-b26c-1dc2a60e3b25-0' usage_metadata={'input_tokens': 24, 'output_tokens': 32, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
